In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import math
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [ ]:
os.chdir('d:\\Users\\Patri\\Documents\\1.TCC\\Notebooks\\WebScrapAcordaosPDF')

In [ ]:
options = webdriver.ChromeOptions()
# options.add_argument("headless")
driver = webdriver.Chrome(executable_path=r"C:\chromedriver\chromedriver.exe", options=options)


In [ ]:
# Webscrapping CARF Precedentes das Súmulas PDfs
inicio = datetime.now()
hora_inicio = inicio.strftime("%H:%M:%S")

for count, num_acordao in enumerate(acordaosabaixar):
    if count <= 1174: continue
    # Carrega página de pesquisa
    driver.get("https://carf.fazenda.gov.br/sincon/public/pages/ConsultarJurisprudencia/consultarJurisprudenciaCarf.jsf")
    driver.find_element(By.CSS_SELECTOR, ".container").click()
    NumAcordao = driver.find_element(By.ID, "valor_pesquisa1")
    NumAcordao.send_keys(num_acordao)
    driver.find_element(By.XPATH, "/html/body/div[1]/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/div[2]/table/tbody/tr/td[2]/label/input").click()
    driver.find_element(By.ID, "botaoPesquisarCarf").click()
    WebDriverWait(driver, 30000).until(EC.presence_of_element_located((By.ID, "tblJurisprudencia:0:numDecisao")))
    driver.find_element(By.ID, "tblJurisprudencia:0:numDecisao").click()

    #   Clicar no botão de download do PDF:
    WebDriverWait(driver, 30000).until(EC.presence_of_element_located((By.XPATH, "//div/a/img")))
    driver.find_element(By.XPATH, "//div/a/img").click()  

    # #   Clicar no botão de Voltar:  
    # driver.find_element(By.XPATH, "//div[3]/input").click() 
    decorrido = datetime.now() - inicio
    expectativa = decorrido / (count + 1) * len(acordaosabaixar)
    if decorrido > expectativa: 
        restante = 0 
    else: 
        restante = expectativa - decorrido
    log_line= str(decorrido)[:-7] + " " + str(expectativa)[:-7] + " " + str(restante)[:-7] + " " + str(num_acordao) + " " + str(count) + "/" + str(len(acordaosabaixar))
    print(log_line)

    # Clicar no botão de Voltar:  
    driver.find_element(By.XPATH, "//div[3]/input").click()   

    # Clicar no botão de Voltar:  
    driver.find_element(By.XPATH, "/html/body/div[1]/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div[2]/input[1]").click()   

In [ ]:
# !pip install bs4

In [ ]:
import requests

page = requests.get("http://idg.carf.fazenda.gov.br/jurisprudencia/sumulas-carf/quadro-geral-de-sumulas-1")
page

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
paragrafos = soup.find_all('p')

In [ ]:
# Rotina para iterar sobre a lista de Súmulas e Precedentes e gerar um DataFrame
sumulaseprecedentes = pd.DataFrame(columns=("Sumula", "TextoSumula", "Precedente"))
precedentes = []
sumula_num = 0
itera_precedentes = False

for count, valor in enumerate(paragrafos):   
    linha = paragrafos[count].get_text()
    linha = linha.replace(u'\xa0', u' ') # Converte non-breaking space em space

    if len(linha) == 0 : continue

    # Identifica o número da súmula
    if linha[:6] == "Súmula" : 
        sumula_num = int(re.findall('(?:Súmula)(?:.*?)(\d*$)', linha)[0])
        vinculante = ""
        continue

    # Identifica o número dos precedentes
    if itera_precedentes == True : 
        if re.search('(Vinculante)', linha) :
            vinculante = re.findall('(?:Vinculante, conforme )(.*)', linha)[0]

        linha = re.sub('(nº )', '', linha)
        linha = re.sub('(Acórdão )', '', linha)
        linha = re.sub('(CSRF/)', '', linha)
        linha = re.sub('(CSRF )', '', linha)
        linha = re.sub('(PLENO )', '', linha)        
        linha = re.sub('(de )', ' ', linha)
        linha = re.sub('(e )', ' ', linha)
        linha = re.sub('(\d+/\d+/\d+)', '', linha)        
        linha = re.sub('(,)', '', linha)
        linha = re.sub('(;)', '', linha)
        linha = re.sub('(\. )', ' ', linha)
        linha = re.sub('(\.$)', '', linha)
        linha = re.sub('( \.)', ' ', linha)     
        linha = re.sub('(- )', '-', linha)
        listaprecedentes = re.findall('(\d*-\d*.\d*?\S)(?:,|;| |\n|$)', linha)
        
        # if count in range (30,50): print(count, sumula_num, linha, listaprecedentes)    
        # if sumula_num in range (161): print(count, sumula_num, linha, listaprecedentes)    
        for item in listaprecedentes:            
            sumulaseprecedentes = sumulaseprecedentes.append({
                "Sumula": sumula_num, 
                "TextoSumula": texto_sum, 
                "Precedente": item,
                "Vinculante": vinculante
                }, ignore_index=True)
        # print ("Quantidade de precedentes:", len(listaprecedentes))

        # Reinicializa a rotina e a lista de precedentes:
        precedentes = []
        itera_precedentes = False
        vinculante = ""
        continue
    
    # Identifica a linha anterior ao número dos precedentes e dispara a rotina "itera_precedentes"
    # if linha[:22] == "Acórdãos Precedentes:" or linha[:22] == "AcórdãosPrecedentes:" : 
    if re.findall('(Acórdãos).*?(Precedentes:)', linha): 
        itera_precedentes = True
        continue

    # Identifica se é súmula vinculante
    if re.search('(Vinculante)', linha) :
        vinculante = re.findall('(?:Vinculante, conforme )(.*)', linha)[0]
        if len (sumulaseprecedentes.query('Sumula == ' + str(sumula_num))):
            sumulaseprecedentes.loc[sumulaseprecedentes['Sumula'] == sumula_num, ['Vinculante']] = vinculante
        # print(vinculante)

    # Captura o texto da súmula
    texto_sum = linha

In [ ]:
acordaosabaixar = sumulaseprecedentes['Precedente'].to_list()

In [ ]:
# 146 - Acórdão '204-00304' É uma Resolução, não um Acórdão
# 277 - Acórdão nº CSRF/04-00.6676, de 19/09/2007 não encontrado - Súmula 39
# 955 - Acórdão nº 3802-000.570, de 05/07/2011 não encontrado - Súmula 126
# Acórdão nº 3202-003.057 não encontrado - Súmula 155
# 03201-002.029, corrigir para 3201-002.029
# 949,3201-002.860 recisou ser corrigido manualmente para 3201-002.860
# 9303-004.949 precisou ser inserido manualmente

# acordaosabaixar('03201-002.029')

# acordaosabaixar.remove('204-00304')
# acordaosabaixar.remove('04-00.6676')
# acordaosabaixar.remove('3802-000.570')
# acordaosabaixar.append('9303-004.949')
# acordaosabaixar.remove('3202-003.057')

# acordaosabaixar[1174]
len(acordaosabaixar)

In [ ]:
sumulaseprecedentes['Sumula'].value_counts().sort_index().sum()

In [ ]:
amostra = sumulaseprecedentes[['Sumula', 'Precedente']]
amostra

In [ ]:
amostra = sumulaseprecedentes[['Sumula', 'Precedente']]
# lista_acordaos = amostra.query("Sumula == [137, 128, 17, 157, 108, 1, 49, 2]")['Precedente'].values
# lista_acordaos
lista_acordaos = amostra.query("Sumula == [2]")['Precedente'].values
lista_acordaos

In [ ]:
#Webscrapping para baixar PDFs de uma súmula expecífica
acordaosabaixar = lista_acordaos

options = webdriver.ChromeOptions()
# options.add_argument("headless")
driver = webdriver.Chrome(executable_path=r"C:\chromedriver\chromedriver.exe", options=options)

# Webscrapping CARF Precedentes das Súmulas PDfs
inicio = datetime.now()
hora_inicio = inicio.strftime("%H:%M:%S")

for count, num_acordao in enumerate(acordaosabaixar):
    # if count <= 1174: continue
    # Carrega página de pesquisa
    driver.get("https://carf.fazenda.gov.br/sincon/public/pages/ConsultarJurisprudencia/consultarJurisprudenciaCarf.jsf")
    driver.find_element(By.CSS_SELECTOR, ".container").click()
    NumAcordao = driver.find_element(By.ID, "valor_pesquisa1")
    NumAcordao.send_keys(num_acordao)
    driver.find_element(By.XPATH, "/html/body/div[1]/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/div[2]/table/tbody/tr/td[2]/label/input").click()
    driver.find_element(By.ID, "botaoPesquisarCarf").click()
    WebDriverWait(driver, 30000).until(EC.presence_of_element_located((By.ID, "tblJurisprudencia:0:numDecisao")))
    driver.find_element(By.ID, "tblJurisprudencia:0:numDecisao").click()

    #   Clicar no botão de download do PDF:
    WebDriverWait(driver, 30000).until(EC.presence_of_element_located((By.XPATH, "//div/a/img")))
    driver.find_element(By.XPATH, "//div/a/img").click()  

    # #   Clicar no botão de Voltar:  
    # driver.find_element(By.XPATH, "//div[3]/input").click() 
    decorrido = datetime.now() - inicio
    expectativa = decorrido / (count + 1) * len(acordaosabaixar)
    if decorrido > expectativa: 
        restante = 0 
    else: 
        restante = expectativa - decorrido
    log_line= str(decorrido)[:-7] + " " + str(expectativa)[:-7] + " " + str(restante)[:-7] + " " + str(num_acordao) + " " + str(count) + "/" + str(len(acordaosabaixar))
    print(log_line)

    # Clicar no botão de Voltar:  
    driver.find_element(By.XPATH, "//div[3]/input").click()   

    # Clicar no botão de Voltar:  
    driver.find_element(By.XPATH, "/html/body/div[1]/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div[2]/input[1]").click()  

In [ ]:
# Webscrapping para pegar o número do Processo a partir do Acórdão

dfA = pd.DataFrame(columns = [
    "numProcesso",
    "numDecisao"
])

options = webdriver.ChromeOptions()
# options.add_argument("headless")
driver = webdriver.Chrome(executable_path=r"C:\chromedriver\chromedriver.exe", options=options)

# Webscrapping CARF Precedentes das Súmulas PDfs
inicio = datetime.now()
hora_inicio = inicio.strftime("%H:%M:%S")

for count, num_acordao in enumerate(lista_acordaos):
    # if count <= 85 : continue
    # Carrega página de pesquisa
    driver.get("https://carf.fazenda.gov.br/sincon/public/pages/ConsultarJurisprudencia/consultarJurisprudenciaCarf.jsf")
    driver.find_element(By.CSS_SELECTOR, ".container").click()
    NumAcordao = driver.find_element(By.ID, "valor_pesquisa1")
    NumAcordao.send_keys(num_acordao)
    driver.find_element(By.XPATH, "/html/body/div[1]/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div/div[2]/div[2]/table/tbody/tr/td[2]/label/input").click()
    driver.find_element(By.ID, "botaoPesquisarCarf").click()
    WebDriverWait(driver, 30000).until(EC.presence_of_element_located((By.ID, "tblJurisprudencia:0:numDecisao")))
    driver.find_element(By.ID, "tblJurisprudencia:0:numDecisao").click()

    # Capturar dados para o dataset
    numProcesso=driver.find_element(By.ID, "formAcordaos:numProcesso").text
    numDecisao=driver.find_element(By.ID, "formAcordaos:numDecisao").text

    # Inserir dados do Acórdão no Dataframe
    dfA = dfA.append({ 
        "numProcesso"  : str(numProcesso), 
        "numDecisao"   : str(numDecisao)
    }, ignore_index=True)
        
    # #   Clicar no botão de download do PDF:
    # WebDriverWait(driver, 30000).until(EC.presence_of_element_located((By.XPATH, "//div/a/img")))
    # driver.find_element(By.XPATH, "//div/a/img").click()  

    # Clicar no botão de Voltar:  
    driver.find_element(By.XPATH, "//div[3]/input").click()   

    # Clicar no botão de Voltar:  
    driver.find_element(By.XPATH, "/html/body/div[1]/table/tbody/tr[2]/td/div/div[3]/div[2]/form/div[2]/input[1]").click()

    print(count)

In [ ]:
os.chdir('d:\\Users\\Patri\\Documents\\1.TCC\\Notebooks\\WebscrapSúmula\\Amostra')

In [ ]:
dfA.to_csv('Procs_Acordaos.csv')

In [ ]:
dfA = pd.read_csv('Procs_Acordaos.csv')

In [ ]:
acordaosabaixar = sumulaseprecedentes['Precedente'].to_list()

In [ ]:
dfA['numDecisao']

In [ ]:
# !pip install pikepdf
import pikepdf

In [ ]:
# Decrypt PDFs to DECRYPT folder

lista = [f for f in os.listdir('.') if os.path.isfile(f)]
len(lista)

if not 'DECRYPT' in os.listdir(): os.mkdir("DECRYPT")

inicio = datetime.now()

for count, arquivo in enumerate(lista):
    filename=lista[count]
    
    with pikepdf.open(filename) as pdf:
        num_pages = len(pdf.pages)
        del pdf.pages[-1]
        pdf.save('DECRYPT\\' + filename)

    decorrido = datetime.now() - inicio
    expectativa = decorrido / (count+1) * len(lista)
    if decorrido > expectativa: 
        restante = 0 
    else: 
        restante = expectativa - decorrido
    print (count, str(decorrido)[:-7], str(expectativa)[:-7], str(restante)[:-7], filename)

In [ ]:
os.chdir('DECRYPT')

In [ ]:
len(os.listdir())

In [ ]:
import pdfminer
# from pdfminer.high_level import extract_text

In [ ]:
# https://towardsdatascience.com/pdf-text-extraction-in-python-5b6ab9e92dd

from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

def convert_pdf_to_string(file_path):

	output_string = StringIO()
	with open(file_path, 'rb') as in_file:
	    parser = PDFParser(in_file)
	    doc = PDFDocument(parser)
	    rsrcmgr = PDFResourceManager()
	    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
	    interpreter = PDFPageInterpreter(rsrcmgr, device)
	    for page in PDFPage.create_pages(doc):
	        interpreter.process_page(page)

	return(output_string.getvalue())

lista = [f for f in os.listdir('.') if os.path.isfile(f)]

inicio = datetime.now()

data_prec = pd.DataFrame(columns = ['Completo'])

for count, arquivo in enumerate(lista):
    textotodo = convert_pdf_to_string(arquivo)
    data_prec = data_prec.append({ 'Completo' : textotodo }, ignore_index=True)

    decorrido = datetime.now() - inicio
    expectativa = decorrido / (count+1) * len(lista)
    if decorrido > expectativa: 
        restante = 0 
    else: 
        restante = expectativa - decorrido

    print (count, str(decorrido)[:-7], str(expectativa)[:-7], str(restante)[:-7], arquivo)

In [ ]:
data_prec.to_csv('acordaos_textocompleto.csv', index=False)

In [ ]:
def extrai_texto_acordao(texto_ac):
    regex_num_proc = '(?:[^\d]|^)(\d{5}\.\d{6}\/\d{4}-\d{2})(?:[^\d]|$)' # obtém um número de processo no formato 00000.000000/0000-00
    regex_num_ac = '(?:[^\d]|^)(?P<acordao>\d{2,4}-\d{2,4}\.\d{3})(?:[^\d]|$)' # obtém um número de acórdão no formato 00-00.000 até 0000-0000.000

    proc_num = list(set(re.findall(regex_num_proc, texto_ac))) # localiza nº processo e exclui repetições
    if len(proc_num) == 1 : proc_num = proc_num[0]
    # print(proc_num)

    acordaos = list(set(re.findall(regex_num_ac, texto_ac))) # localiza nº processo e exclui repetições
    # print(acordaos)

    texto_ac = re.sub('\n', ' ', texto_ac)
    # print(texto_ac)
    # pyperclip.copy(texto_ac)

    regex_relatorio = '(?:Relatório)(.*?)(?:Voto)'
    try:
        relatorio = re.findall(regex_relatorio, texto_ac, re.IGNORECASE)[0]
        max(len(re) for re in relatorio)
    except: 
        relatorio = "Não encontrado"
    # print (relatorio)

    regex_voto = '(?:Voto)(.*?)($)'
    try:
        voto = re.findall(regex_voto, texto_ac, re.IGNORECASE)[0]
        max(len(vo) for vo in voto)
    except:
        voto = "Não encontrado"
    # print (voto)

    colunas = ['Processo', 'Decisao', 'Relatorio', 'Voto']
    dados = [proc_num, acordaos, relatorio, voto]
    inserir = {'Processo': proc_num, 'Decisao':list(acordaos), 'Relatorio':relatorio, 'Voto':voto}

    db_acordao = pd.DataFrame(columns = colunas)
    db_acordao = db_acordao.append(inserir, ignore_index=True)

    return db_acordao

In [ ]:
colunas = ['Processo', 'Decisao', 'Relatorio', 'Voto']

db_acordao = pd.DataFrame(columns = colunas)

for i in range(1,len(data_prec)-1):
    if i in [] : continue
    # print(i)
    db_acordao2 = extrai_texto_acordao(data_prec['Completo'][i])
    frames = [db_acordao, db_acordao2]
    db_acordao = pd.concat(frames)

db_acordao

In [ ]:
# os.listdir()
# os.getcwd()
os.chdir('d:\\Users\\Patri\\Documents\\1.TCC\\Notebooks\\WebscrapSúmula\\PDF Súmulas\\XML\\Acordao')

In [ ]:
dataset_precedentes = pd.read_csv('dataset_precedentes.csv')

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

df = db_acordao

# Recebe um texto ou series e retorna os tokens
def tokeniza(texto):
    if type(texto) == "Series":
        acordao_text = texto.to_string()
    else:
        acordao_text = str(texto)
    acordao_text = acordao_text.lower() # coloca tudo em minúsculo
    acordao_words = RegexpTokenizer(r'\w+') # remove pontuação ao selecionar apenas palavras
    acordao_tokens = acordao_words.tokenize(acordao_text) # tokeniza (separa as palavras)

    # Extrair Stopwords em Português
    pt_stops = set(stopwords.words('portuguese'))
    pt_stops.update(['fls', 'cuida', 'nº', 'º', 'ª', 'n', 'fl'])
    words = (acordao_tokens)
    lista_words = [word for word in words if word not in pt_stops] # remove stopwords
    if not len(lista_words): lista_words = []
    return lista_words 

# Remove números de um texto
def remove_numeros(texto):
    output = ''.join(c for c in texto if not c.isdigit())
    return output

def lemma(texto):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(texto)


# Tokeniza trechos do acórdão e salva tokens em uma nova coluna
colunas=['Voto', 'Relatorio']

for coluna in colunas:
    df['t_'+coluna] = df[coluna].apply(tokeniza)

coluna = 'Relatorio'
# df['t0_'+coluna] = df[coluna].apply(tokeniza(remove_numeros))

In [ ]:
df

In [ ]:
df.insert(0,'S057VO', 1) # adiciona coluna rotulando como True para a Sùmula 57

In [ ]:
df

In [ ]:
dataset_precedentes = df[['Relatorio', 'S057VO']]

In [ ]:
dataset_precedentes

In [ ]:
path = 'd:\\Users\\Patri\\Documents\\1.TCC\\Notebooks\\WebScrapAcordaosPDF\\NLTK'
dataset_precedentes.to_csv(path+'\\dataset_precedentes.csv', index=False)